# 📖 NCTB Class 3 Bengali Book — OCR Pipeline
### EasyOCR দিয়ে Scanned Bengali PDF → Unicode Text

**কেন EasyOCR?**
- NCTB Class 3 বাংলা বইটি scanned image PDF
- Standard Tesseract Bengali Unicode output দিতে পারে না এই ধরনের PDF এ
- EasyOCR deep learning based — বাংলার জন্য অনেক বেশি accurate

**Steps:**
1. PDF → Page images (pdftoppm)
2. EasyOCR দিয়ে Bengali text extract
3. Clean ও chunk করা
4. FAISS vector index তৈরি
5. Test retrieval

---
⚠️ **Setup:** PDF টা Kaggle Dataset হিসেবে add করো।  
Path হবে: `/kaggle/input/nctb-class3/Class-3__Bangla_combine__compressed.pdf`

## Step 1: Install Libraries

In [9]:
# Core installs
!pip install -q easyocr
!pip install -q langchain langchain-community
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!apt-get install -q poppler-utils   # pdftoppm এর জন্য

print('✅ সব install হয়েছে!')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.47.0 which is incompatible.
google-colab 1.0.0 requires jupyter-server==2.14.0, but you have jupyter-server 2.12.5 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
gradio 5.49.1 requires pydanti

## Step 2: Setup ও PDF Check

In [10]:
import os, subprocess, json, pickle
import numpy as np
from pathlib import Path

# ── Paths ──────────────────────────────────────────────────────
# PDF_PATH  = '/kaggle/input/nctb-class3/Class-3__Bangla_combine__compressed.pdf'
PDF_PATH = '/kaggle/input/datasets/sharbanichowdhury/nctb-class3/Class-3 Bangla combine _compressed.pdf'
WORK_DIR  = '/kaggle/working'
IMG_DIR   = f'{WORK_DIR}/pages'      # PDF pages as PNG
TEXT_DIR  = f'{WORK_DIR}/texts'      # Extracted text per page
INDEX_DIR = f'{WORK_DIR}/faiss_index'  # Final FAISS index

for d in [IMG_DIR, TEXT_DIR, INDEX_DIR]:
    os.makedirs(d, exist_ok=True)

# PDF exists check
if not os.path.exists(PDF_PATH):
    # Kaggle input এ না পেলে working directory তে খোঁজো
    alt = '/kaggle/working/Class-3__Bangla_combine__compressed.pdf'
    if os.path.exists(alt):
        PDF_PATH = alt
    else:
        raise FileNotFoundError(
            f'PDF পাওয়া যায়নি!\n'
            f'Kaggle Dataset এ PDF add করো এবং path ঠিক করো।\n'
            f'Expected: {PDF_PATH}'
        )

# Page count
result = subprocess.run(
    ['pdfinfo', PDF_PATH], capture_output=True, text=True
)
for line in result.stdout.split('\n'):
    if 'Pages' in line:
        TOTAL_PAGES = int(line.split(':')[1].strip())
        break

print(f'✅ PDF found: {PDF_PATH}')
print(f'📄 মোট পৃষ্ঠা: {TOTAL_PAGES}')
print(f'🗂️  Working dir: {WORK_DIR}')

✅ PDF found: /kaggle/input/datasets/sharbanichowdhury/nctb-class3/Class-3 Bangla combine _compressed.pdf
📄 মোট পৃষ্ঠা: 110
🗂️  Working dir: /kaggle/working


## Step 3: PDF → Page Images (High DPI)

In [11]:
from tqdm import tqdm

# ── Config ────────────────────────────────────────────────────
DPI         = 300    # High DPI = better OCR accuracy
START_PAGE  = 4      # প্রথম ৩ পৃষ্ঠা cover/blank, skip করি
END_PAGE    = TOTAL_PAGES
BATCH_SIZE  = 10     # কতটা page একসাথে convert করবে

def convert_batch(start, end, dpi=DPI):
    """PDF এর start থেকে end পৃষ্ঠা PNG তে convert করা।"""
    result = subprocess.run([
        'pdftoppm', '-png', '-r', str(dpi),
        '-f', str(start), '-l', str(end),
        PDF_PATH, f'{IMG_DIR}/page'
    ], capture_output=True)
    return result.returncode == 0

# Convert all pages in batches
print(f'🔄 Converting pages {START_PAGE}–{END_PAGE} to PNG @ {DPI} DPI...')
print(f'   (এটা ৫-১০ মিনিট সময় নিতে পারে)')

batches = range(START_PAGE, END_PAGE + 1, BATCH_SIZE)
for batch_start in tqdm(batches, desc='Converting batches'):
    batch_end = min(batch_start + BATCH_SIZE - 1, END_PAGE)
    convert_batch(batch_start, batch_end)

# Count converted images
pages = sorted(Path(IMG_DIR).glob('*.png'))
print(f'\n✅ {len(pages)} টি page image তৈরি হয়েছে।')
print(f'   Sample: {pages[0].name} ... {pages[-1].name}')

🔄 Converting pages 4–110 to PNG @ 300 DPI...
   (এটা ৫-১০ মিনিট সময় নিতে পারে)


Converting batches: 100%|██████████| 11/11 [01:57<00:00, 10.70s/it]


✅ 107 টি page image তৈরি হয়েছে।
   Sample: page-004.png ... page-110.png


## Step 4: EasyOCR Reader Initialize

In [12]:
import easyocr
import torch

print(f'GPU available: {torch.cuda.is_available()}')
print('🔄 EasyOCR model load হচ্ছে... (প্রথমবার একটু সময় লাগবে)')

# Bengali + English দুটোই recognize করবে
# gpu=True → Kaggle T4 GPU ব্যবহার করবে (অনেক faster)
READER = easyocr.Reader(
    ['bn', 'en'],           # Bengali + English
    gpu=torch.cuda.is_available(),
    verbose=False
)

print('✅ EasyOCR ready!')

# Quick test on first content page
test_img = str(pages[0])
test_result = READER.readtext(test_img, detail=0, paragraph=True)
print(f'\n🧪 Quick test on {pages[0].name}:')
print('\n'.join(test_result[:5]))

GPU available: True
🔄 EasyOCR model load হচ্ছে... (প্রথমবার একটু সময় লাগবে)


✅ EasyOCR ready!

🧪 Quick test on page-004.png:
আমার বাংলা বই তৃতীয় শ্রেণি
Iিh


## Step 5: Full Book OCR

পুরো বইয়ের সব পৃষ্ঠা OCR করা হবে।  
প্রতিটি পৃষ্ঠার text আলাদা file এ save হবে — resume করতে সুবিধা হবে।

In [13]:
import unicodedata
import re

def clean_bengali(text):
    """OCR output clean করা।"""
    text = unicodedata.normalize('NFC', text)  # Unicode normalize
    text = re.sub(r'\n{3,}', '\n\n', text)     # Extra blank lines সরানো
    text = re.sub(r'[ \t]+', ' ', text)         # Extra spaces সরানো
    return text.strip()

def ocr_page(img_path):
    """একটি page এর OCR করা।"""
    results = READER.readtext(
        str(img_path),
        detail=0,
        paragraph=True,       # Text কে paragraphs এ group করে
        width_ths=0.7,        # Word grouping threshold
        decoder='greedy'
    )
    return clean_bengali('\n'.join(results))

# ── Run OCR on all pages ───────────────────────────────────────
all_page_texts = {}  # {page_num: text}
failed_pages = []

print(f'🔄 {len(pages)} পৃষ্ঠার OCR শুরু হচ্ছে...')
print('   (প্রতি পৃষ্ঠায় ~৩-৫ সেকেন্ড, মোট ~১০ মিনিট)')
print('   Text file গুলো automatically save হতে থাকবে।\n')

for img_path in tqdm(pages, desc='OCR Progress'):
    # Page number extract করা filename থেকে
    page_num = int(img_path.stem.split('-')[-1])
    text_file = Path(TEXT_DIR) / f'page_{page_num:03d}.txt'
    
    # Already processed? Skip (resume support)
    if text_file.exists():
        with open(text_file, encoding='utf-8') as f:
            all_page_texts[page_num] = f.read()
        continue
    
    try:
        text = ocr_page(img_path)
        all_page_texts[page_num] = text
        
        # Save immediately (session crash হলেও progress নষ্ট হবে না)
        with open(text_file, 'w', encoding='utf-8') as f:
            f.write(text)
            
    except Exception as e:
        print(f'\n⚠️ Page {page_num} failed: {e}')
        failed_pages.append(page_num)
        all_page_texts[page_num] = ''

# Summary
total_words = sum(len(t.split()) for t in all_page_texts.values())
bengali_chars = sum(
    sum(1 for c in t if '\u0980' <= c <= '\u09FF')
    for t in all_page_texts.values()
)

print(f'\n✅ OCR Complete!')
print(f'   Pages processed: {len(all_page_texts)}')
print(f'   Total words: {total_words:,}')
print(f'   Bengali Unicode chars: {bengali_chars:,}')
if failed_pages:
    print(f'   ⚠️ Failed pages: {failed_pages}')

🔄 107 পৃষ্ঠার OCR শুরু হচ্ছে...
   (প্রতি পৃষ্ঠায় ~৩-৫ সেকেন্ড, মোট ~১০ মিনিট)
   Text file গুলো automatically save হতে থাকবে।



OCR Progress: 100%|██████████| 107/107 [05:07<00:00,  2.88s/it]


✅ OCR Complete!
   Pages processed: 107
   Total words: 10,971
   Bengali Unicode chars: 45,230


## Step 6: Sample দেখা

In [14]:
# কয়েকটা page এর text দেখো
print('=' * 60)
print('📄 SAMPLE OUTPUT — OCR করা Bengali Text:')
print('=' * 60)

# First few non-empty pages
shown = 0
for pg, text in sorted(all_page_texts.items()):
    if text.strip() and len(text.split()) > 10:
        print(f'\n[পৃষ্ঠা {pg}]')
        print(text[:400])
        print('---')
        shown += 1
        if shown >= 3:
            break

# Bengali word count check
sample_text = list(all_page_texts.values())[5]
bn_chars = sum(1 for c in sample_text if '\u0980' <= c <= '\u09FF')
total_chars = len(sample_text)
print(f'\n📊 Bengali char ratio: {bn_chars}/{total_chars} = {bn_chars/max(total_chars,1)*100:.1f}%')

if bn_chars / max(total_chars, 1) > 0.3:
    print('✅ Bengali text ঠিকমতো extract হয়েছে!')
else:
    print('⚠️ Bengali chars কম — OCR quality check করো')

📄 SAMPLE OUTPUT — OCR করা Bengali Text:

[পৃষ্ঠা 6]
জাতীয় শিক্ষাক্রম ও পাঠ্যপুস্তক বোর্ড কর্তৃক ২০২৪ শিক্ষাবর্ষ থেকে তৃতীয় শ্রেণির পাঠ্যপুস্তকরূপে নির্ধারিত
আমার বাংলা বই তৃতীয় শ্রেণি
শ্ঁুঁ টিঁ ন জাতীয় শিক্ষাক্রম ও পাঠ্যপুস্তক বোর্ড , বাংলাদেশ বংলাদেশ
---

[পৃষ্ঠা 7]
জাতীয় শিক্ষাক্রম ও পাঠ্যপুস্তক বোর্ড , 9 বাংলাদেশ ৬৯-৭০ , মতিঝিল বাণিজ্যিক এলাকা, ঢাকা ১০০০ কর্তৃক প্রকাশিত (প্রকাশক কর্তৃক সর্বস্বত্ব সংরক্ষিত) প্রথম সংস্করণ সংকলন ও রচনা অধ্যাপক ড শোয়াইব জিবরান অধ্যাপক ড সুমন সাজ্জাদ অধ্যাপক ড তারিক মনজুর মোহাম্মদ মামুন অর রশীদ মোঃ মাহমুদুল হাসান খুরশীদা আক্তার জাহান মোঃ আব্দুল মুমিন মোছাব্বির
শিল্প নির্দেশনা হাশেম খান
ছবি ও অলংকরণ সাজ্জাদ মজুমদার মোঃ মহিদুল হ
---

[পৃষ্ঠা 8]
প্রসঙ্গকথা
প্রাথমিক স্তর শিক্ষার ভিত্তিভূমি। প্রাথমিক শিক্ষা সুনির্দিষ্ট লক্ষ্যমুখী ও পরিকল্পিত না হলে গোটা শিক্ষাব্যবস্থাই দুর্বল হয়ে পড়ে| এই বিষয়টি বিবেচনায় নিয়ে ২০১০ সালের শিক্ষানীতিতে প্রাথমিক স্তরকে বিশেষ গুরুত্ব দেওয়া হয়েছে | বিশ্বের উন্নত দেশসমূহের সাথে সংগতি রেখে প্রাথমিক স্তরের পরিসর বৃদ্ধি এবং

## Step 7: Full Book Text তৈরি + Chunking

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# সব page text একসাথে জোড়া লাগানো
FULL_TEXT = ''
for pg_num in sorted(all_page_texts.keys()):
    text = all_page_texts[pg_num]
    if text.strip():
        FULL_TEXT += f'\n\n--- পৃষ্ঠা {pg_num} ---\n{text}'

# Full text save করো
with open(f'{WORK_DIR}/class3_bangla_full.txt', 'w', encoding='utf-8') as f:
    f.write(FULL_TEXT)
print(f'✅ Full text saved: {len(FULL_TEXT):,} characters, {len(FULL_TEXT.split()):,} words')

# ── Chunking ──────────────────────────────────────────────────
# Bengali-aware: '।' (দাড়ি) কে separator হিসেবে ব্যবহার করা হয়েছে
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=75,
    separators=['\n\n', '\n', '।', '.', ' ', '']
)

CHUNKS = splitter.split_text(FULL_TEXT)

# Empty chunks filter করো
CHUNKS = [c.strip() for c in CHUNKS if len(c.strip().split()) >= 5]

print(f'📦 মোট chunks: {len(CHUNKS)}')
print(f'   Average chunk size: {sum(len(c) for c in CHUNKS) // len(CHUNKS)} chars')
print(f'\n🔍 Sample chunk:')
print(CHUNKS[10] if len(CHUNKS) > 10 else CHUNKS[0])

✅ Full text saved: 60,159 characters, 11,399 words
📦 মোট chunks: 190
   Average chunk size: 331 chars

🔍 Sample chunk:
কৃতজ্ঞতা জানাই| কৃতজ্ঞতা জানাই তাঁদের প্রতিও যাঁরা অলংকরণের মাধ্যমে বইটিকে শিশুদের জন্যে চিত্তাকর্ষক করে তুলেছেন | ২০২৪ সালের পরিবর্তিত পরিস্থিতিতে প্রয়োজনের নিরিখে পাঠ্যপুস্তকসমূহ পরিমার্জন করা হয়েছে সতর্কতা সত্ত্বেও কিছু ভুলত্রুটি থেকে যেতে পারে/ সুধিজনের কাছ থেকে যৌক্তিক পরামর্শ ও নির্দেশনা পেলে সেগুলো গুরুত্বের সাথে বিবেচনায় নেওয়া হবেI পরিশেষে বইটি যাদের জন্য, সেই কোমলমতি শিক্ষার্থীদের সার্বিক কল্যাণ কামনা করছি


## Step 8: Embeddings + FAISS Index

In [16]:
from sentence_transformers import SentenceTransformer
import faiss
import torch

print('🔄 Embedding model load হচ্ছে...')

# Multilingual model — Bengali, English উভয়ই support করে
EMBED_MODEL = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

print(f'✅ Model loaded on: {"GPU" if torch.cuda.is_available() else "CPU"}')
print(f'🔄 {len(CHUNKS)} chunks embed করা হচ্ছে...')

EMBEDDINGS = EMBED_MODEL.encode(
    CHUNKS,
    batch_size=128,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print(f'\n✅ Embeddings shape: {EMBEDDINGS.shape}')

# ── FAISS Index ───────────────────────────────────────────────
dim = EMBEDDINGS.shape[1]  # 384
INDEX = faiss.IndexFlatIP(dim)  # Inner Product = Cosine similarity
INDEX.add(EMBEDDINGS.astype('float32'))

# Save everything
faiss.write_index(INDEX, f'{INDEX_DIR}/bangla_class3.faiss')
with open(f'{INDEX_DIR}/chunks.pkl', 'wb') as f:
    pickle.dump(CHUNKS, f)

print(f'\n💾 FAISS index saved!')
print(f'   Vectors stored: {INDEX.ntotal}')
print(f'   Files:')
print(f'   - {INDEX_DIR}/bangla_class3.faiss')
print(f'   - {INDEX_DIR}/chunks.pkl')

2026-02-19 06:33:42.148152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771482822.566133      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771482822.677188      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771482823.662595      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771482823.662666      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771482823.662669      55 computation_placer.cc:177] computation placer alr

🔄 Embedding model load হচ্ছে...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Model loaded on: GPU
🔄 190 chunks embed করা হচ্ছে...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]


✅ Embeddings shape: (190, 384)

💾 FAISS index saved!
   Vectors stored: 190
   Files:
   - /kaggle/working/faiss_index/bangla_class3.faiss
   - /kaggle/working/faiss_index/chunks.pkl


## Step 9: Retrieval Test

In [17]:
def retrieve(question, k=3):
    """Student এর question এর জন্য NCTB content খুঁজে আনা।"""
    q_emb = EMBED_MODEL.encode(
        [question], normalize_embeddings=True, convert_to_numpy=True
    ).astype('float32')
    scores, indices = INDEX.search(q_emb, k)
    return [
        {'text': CHUNKS[i], 'score': float(s)}
        for s, i in zip(scores[0], indices[0])
        if i >= 0
    ]

# Test questions — Bengali ও English
TEST_QS = [
    'তোমার বাড়িতে কতজন আছে?',
    'বন্ধু মানে কী?',
    'What is this lesson about?',
    'পরিবারের সদস্য কারা?',
    'ধন্যবাদ কাকে বলি?'
]

print('=' * 60)
print('🧪 RETRIEVAL TEST')
print('=' * 60)

for q in TEST_QS:
    print(f'\n❓ প্রশ্ন: {q}')
    results = retrieve(q, k=2)
    for i, r in enumerate(results, 1):
        preview = r['text'][:200].replace('\n', ' ')
        print(f'   [{i}] Score={r["score"]:.3f}: {preview}...')
    print('-' * 40)

🧪 RETRIEVAL TEST

❓ প্রশ্ন: তোমার বাড়িতে কতজন আছে?
   [1] Score=0.667: এবার একটা ছক আঁকি | ছকটিতে নিজের ভালো লাগার কথা লিখি স্যার বোর্ডে একটি ছক আঁকলেন বললেন , আমার মতো করে তোমরাও ছকটি আঁকো...
   [2] Score=0.657: চোখে পড়ল কুমারপাড়া নৌকায় উঠলেন বাবার বন্ধু মধু পাল ! তিনি ? মাটি দিয়ে শখের হাঁড়ি বানান রঙিন হাঁড়িগুলো দেখতে খুব সুন্দর মধু কাকা তিথিকে ৫ রঙিন হাঁড়ি দিলেন বললেন , বাসায় সাজিয়ে রেখো...
----------------------------------------

❓ প্রশ্ন: বন্ধু মানে কী?
   [1] Score=0.561: জিজ্ঞেস করলেন তুমি কী করবে? রাশেদ বলল ? অঙ্ক দৌড় ও মোরগ লড়াই করব ক্লাসের সবাই ভাবছিল , রাশেদ পারবে তো ! নোমান স্যার বললেন , খুব ভালো লাগল রাশেদ gূ...
   [2] Score=0.509: --- পৃষ্ঠা 35 --- পাঠ ব্যাঙের সাজা...
----------------------------------------

❓ প্রশ্ন: What is this lesson about?
   [1] Score=0.489: হয়েছে | এর মাধ্যমে শিক্ষাকে অধিকতর জীবনমুখী ও ফলপ্রসূ করার প্রয়াস বাস্তব ভিত্তি পেয়েছে | বিশ্বায়নের বাস্তবতায় শিশুদের মনোজাগতিক অবস্থাকেও শিক্ষাক্রমে বিশেষভাবে বিবেচনায় রাখা হয়েছে শিক্ষাক্রম 

## Step 10: Phi-3 RAG Prompt Template

In [18]:
def build_rag_prompt(student_question, k=3, mode='chat'):
    """
    Phi-3 এর জন্য complete RAG prompt।
    mode: 'chat' = answer দেওয়া, 'quiz' = MCQ generate করা
    """
    # Relevant context retrieve করো
    chunks = retrieve(student_question, k=k)
    context_text = '\n\n'.join(
        f'[অংশ {i+1}]:\n{c["text"]}' for i, c in enumerate(chunks)
    )
    
    if mode == 'chat':
        system = (
            'তুমি একটি AI শিক্ষক। বাংলাদেশের তৃতীয় শ্রেণির ছাত্রছাত্রীদের সাহায্য করো।\n'
            'শুধুমাত্র নিচের NCTB পাঠ্যবইয়ের অংশ ব্যবহার করে সহজ বাংলায় উত্তর দাও।\n'
            'উত্তর ছোট ও বন্ধুত্বপূর্ণ রাখো যেন একটি ৮-৯ বছরের বাচ্চা বুঝতে পারে।'
        )
        user = f'পাঠ্যবইয়ের অংশ:\n{context_text}\n\nছাত্রের প্রশ্ন: {student_question}'
        
    elif mode == 'quiz':
        system = (
            'তুমি একজন শিক্ষক। NCTB পাঠ্যবইয়ের তথ্য থেকে MCQ তৈরি করো।\n'
            'Output শুধু JSON format এ দাও।'
        )
        user = (
            f'পাঠ্যবইয়ের অংশ:\n{context_text}\n\n'
            f'"{student_question}" বিষয়ে ৩টি সহজ MCQ প্রশ্ন তৈরি করো।\n'
            'Format: {"questions": [{"q": "...", "options": ["ক) ...", "খ) ...", "গ) ...", "ঘ) ..."], "answer": "ক"}]}'
        )
    
    # Phi-3 prompt format
    prompt = f'<|system|>\n{system}\n<|end|>\n<|user|>\n{user}\n<|end|>\n<|assistant|>'
    return prompt

# Demo
sample_prompt = build_rag_prompt('বন্ধুত্ব মানে কী?', mode='chat')
print('📝 Sample Chat Prompt (Phi-3 ready):')
print('=' * 60)
print(sample_prompt[:800])
print('...')
print('=' * 60)

quiz_prompt = build_rag_prompt('পরিবার', mode='quiz')
print('\n📝 Sample Quiz Prompt:')
print('=' * 60)
print(quiz_prompt[:600])
print('...')

📝 Sample Chat Prompt (Phi-3 ready):
<|system|>
তুমি একটি AI শিক্ষক। বাংলাদেশের তৃতীয় শ্রেণির ছাত্রছাত্রীদের সাহায্য করো।
শুধুমাত্র নিচের NCTB পাঠ্যবইয়ের অংশ ব্যবহার করে সহজ বাংলায় উত্তর দাও।
উত্তর ছোট ও বন্ধুত্বপূর্ণ রাখো যেন একটি ৮-৯ বছরের বাচ্চা বুঝতে পারে।
<|end|>
<|user|>
পাঠ্যবইয়ের অংশ:
[অংশ 1]:
--- পৃষ্ঠা 35 ---
পাঠ
ব্যাঙের সাজা

[অংশ 2]:
--- পৃষ্ঠা 41 ---
পাঠ ১১
আনন্দের দিন

[অংশ 3]:
জিজ্ঞেস করলেন তুমি কী করবে? রাশেদ বলল ? অঙ্ক দৌড় ও মোরগ লড়াই করব ক্লাসের সবাই ভাবছিল , রাশেদ পারবে তো ! নোমান স্যার বললেন , খুব ভালো লাগল রাশেদ gূ

ছাত্রের প্রশ্ন: বন্ধুত্ব মানে কী?
<|end|>
<|assistant|>
...

📝 Sample Quiz Prompt:
<|system|>
তুমি একজন শিক্ষক। NCTB পাঠ্যবইয়ের তথ্য থেকে MCQ তৈরি করো।
Output শুধু JSON format এ দাও।
<|end|>
<|user|>
পাঠ্যবইয়ের অংশ:
[অংশ 1]:
--- পৃষ্ঠা 8 ---
প্রসঙ্গকথা

[অংশ 2]:
--- পৃষ্ঠা 35 ---
পাঠ
ব্যাঙের সাজা

[অংশ 3]:
--- পৃষ্ঠা 106 ---
পাঠ

"পরিবার" বিষয়ে ৩টি সহজ MCQ প্রশ্ন তৈরি করো।
Format: {"questions": [{"q": "...", "options": ["ক) ...", "খ) ...", "গ) .

## ✅ Summary

In [19]:
import os

faiss_size = os.path.getsize(f'{INDEX_DIR}/bangla_class3.faiss') / 1024
chunks_size = os.path.getsize(f'{INDEX_DIR}/chunks.pkl') / 1024

print('=' * 60)
print('✅ NCTB Class 3 Bengali — RAG Pipeline Complete!')
print('=' * 60)
print(f'''
📊 Stats:
   Total pages OCR'd : {len(all_page_texts)}
   Total words       : {len(FULL_TEXT.split()):,}
   Total chunks      : {len(CHUNKS)}
   FAISS vectors     : {INDEX.ntotal}
   Embedding dim     : {EMBEDDINGS.shape[1]}

💾 Saved Files:
   {WORK_DIR}/class3_bangla_full.txt  ({os.path.getsize(f"{WORK_DIR}/class3_bangla_full.txt")//1024} KB)
   {INDEX_DIR}/bangla_class3.faiss    ({faiss_size:.0f} KB)
   {INDEX_DIR}/chunks.pkl             ({chunks_size:.0f} KB)
   {TEXT_DIR}/page_*.txt              (per-page backups)

🚀 পরবর্তী Notebook:
   → Phi-3 Mini inference (ollama / transformers)
   → Whisper STT: Bengali audio → text
   → Edge-TTS: text → Bengali voice
   → Django API এ integrate করা
''')
print('=' * 60)

✅ NCTB Class 3 Bengali — RAG Pipeline Complete!

📊 Stats:
   Total pages OCR'd : 107
   Total words       : 11,399
   Total chunks      : 190
   FAISS vectors     : 190
   Embedding dim     : 384

💾 Saved Files:
   /kaggle/working/class3_bangla_full.txt  (148 KB)
   /kaggle/working/faiss_index/bangla_class3.faiss    (285 KB)
   /kaggle/working/faiss_index/chunks.pkl             (157 KB)
   /kaggle/working/texts/page_*.txt              (per-page backups)

🚀 পরবর্তী Notebook:
   → Phi-3 Mini inference (ollama / transformers)
   → Whisper STT: Bengali audio → text
   → Edge-TTS: text → Bengali voice
   → Django API এ integrate করা



In [20]:
# Sample text দেখো
with open('/kaggle/working/class3_bangla_full.txt', encoding='utf-8') as f:
    text = f.read()

# Bengali char ratio
bn = sum(1 for c in text if '\u0980' <= c <= '\u09FF')
print(f"Bengali chars: {bn:,} / {len(text):,} = {bn/len(text)*100:.1f}%")
print("\n--- Sample (500 chars) ---")
print(text[500:1000])

Bengali chars: 45,872 / 60,159 = 76.3%

--- Sample (500 chars) ---
চনা অধ্যাপক ড শোয়াইব জিবরান অধ্যাপক ড সুমন সাজ্জাদ অধ্যাপক ড তারিক মনজুর মোহাম্মদ মামুন অর রশীদ মোঃ মাহমুদুল হাসান খুরশীদা আক্তার জাহান মোঃ আব্দুল মুমিন মোছাব্বির
শিল্প নির্দেশনা হাশেম খান
ছবি ও অলংকরণ সাজ্জাদ মজুমদার মোঃ মহিদুল হাসান জয়ন্ত সরকার জন
প্রথম মুদ্রণ: অক্টোবর ২০২৩ পরিমার্জিত সংস্করণ: অক্টোবর ২০২৪ পুনর্মুদ্রণ: সেপ্টেম্বর ২০২৫
ডিজাইন
জাতীয় শিক্ষাক্রম ও পাঠ্যপুস্তক বোর্ড , বাংলাদেশ
গণপ্রজাতন্ত্রী বাংলাদেশ সরকার কর্তৃক বিনামূল্যে বিতরণের জন্য
মুদ্রণে:

--- পৃষ্ঠা 8 ---
প্রসঙ্গকথা
প্রা


In [21]:
import os

for root, dirs, files in os.walk('/kaggle/working'):
    for f in files:
        path = os.path.join(root, f)
        size = os.path.getsize(path) // 1024
        print(f"{path} ({size} KB)")

/kaggle/working/class3_bangla_full.txt (148 KB)
/kaggle/working/pages/page-079.png (352 KB)
/kaggle/working/pages/page-081.png (842 KB)
/kaggle/working/pages/page-023.png (227 KB)
/kaggle/working/pages/page-018.png (836 KB)
/kaggle/working/pages/page-080.png (487 KB)
/kaggle/working/pages/page-105.png (267 KB)
/kaggle/working/pages/page-094.png (480 KB)
/kaggle/working/pages/page-026.png (238 KB)
/kaggle/working/pages/page-007.png (412 KB)
/kaggle/working/pages/page-060.png (513 KB)
/kaggle/working/pages/page-082.png (539 KB)
/kaggle/working/pages/page-062.png (381 KB)
/kaggle/working/pages/page-024.png (268 KB)
/kaggle/working/pages/page-035.png (811 KB)
/kaggle/working/pages/page-077.png (287 KB)
/kaggle/working/pages/page-106.png (458 KB)
/kaggle/working/pages/page-085.png (831 KB)
/kaggle/working/pages/page-083.png (470 KB)
/kaggle/working/pages/page-102.png (592 KB)
/kaggle/working/pages/page-092.png (668 KB)
/kaggle/working/pages/page-071.png (687 KB)
/kaggle/working/pages/page-0

In [22]:
# Files zip করে download করো
import shutil

shutil.make_archive(
    '/kaggle/working/nctb_rag_output',  # zip filename
    'zip',
    '/kaggle/working'                    # কোন folder zip করবে
)
print("✅ ZIP ready! Output tab থেকে download করো।")

✅ ZIP ready! Output tab থেকে download করো।
